In [2]:
# 找寻连续或者独立的序列：
topic_utter_ids = [8,12,13,14,15,48,56]
start_ids = []
end_ids = []
# 一个队列结构的list用于输出连续性utterance id
queue = []
for i, idx in enumerate(topic_utter_ids):
    # 初始化第一个元素：
    if i == 0: 
        queue.append(idx)
    else:
        if queue[-1]+1 == idx:
            queue.append(idx)
        else:
            start_ids.append(queue[0])
            end_ids.append(queue[-1])
            queue = []
            queue.append(idx)
start_ids.append(queue[0])
end_ids.append(queue[-1])

In [3]:
start_ids, end_ids

([8, 12, 48, 56], [8, 15, 48, 56])

In [8]:
cur_pair = []
target_shift = 20
for s,e in zip(start_ids,end_ids):
    cur_pair.extend([s+target_shift,e+target_shift])

SyntaxError: invalid syntax (<ipython-input-8-116f3d0e30e1>, line 4)

In [6]:
cur_pair

[28, 28, 32, 35, 68, 68, 76, 76]

In [5]:
text

' 我  真  是   服  了 '

In [7]:
from transformers import BertModel, BertConfig
model = BertModel.from_pretrained('../../pretrained/bert_base_chinese')
num_tokens, _ = model.embeddings.word_embeddings.weight.shape
model.resize_token_embeddings(21190)
# model.embeddings.word_embeddings.weight.data[21189]
model.embeddings.word_embeddings.weight.data[21188][0]

Some weights of the model checkpoint at ../../pretrained/bert_base_chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor(-0.0413)

In [6]:
model.embeddings.word_embeddings.weight.data[21189][0]

tensor(0.0060)

In [8]:
num_tokens

21128

In [15]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HugginFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Tokenization classes."""

from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import logging
import os
import unicodedata
from io import open

from transformers import cached_path

logger = logging.getLogger(__name__)

# PRETRAINED_VOCAB_ARCHIVE_MAP = {
#     'bert-base-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt",
#     'bert-large-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt",
#     'bert-base-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt",
#     'bert-large-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt",
#     'bert-base-multilingual-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-vocab.txt",
#     'bert-base-multilingual-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt",
#     'bert-base-chinese': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt",
# }
PRETRAINED_VOCAB_POSITIONAL_EMBEDDINGS_SIZE_MAP = {
    'bert-base-uncased': 512,
    'bert-large-uncased': 512,
    'bert-base-cased': 512,
    'bert-large-cased': 512,
    'bert-base-multilingual-uncased': 512,
    'bert-base-multilingual-cased': 512,
    'bert-base-chinese': 512,
}
VOCAB_NAME = 'vocab.txt'


def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    index = 0
    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab  # collections.OrderedDict


def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a peice of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens  # list


class BertTokenizer(object):
    """Runs end-to-end tokenization: punctuation splitting + wordpiece"""

    def __init__(self, vocab_file, do_lower_case=True, mapping=None, max_len=None,
                 never_split=("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]", "[unused98]", "[unused99]", "[unused1]", "[unused2]", "[unused3]", "[unused4]", "[unused5]", "[unused6]")):

        if not os.path.isfile(vocab_file):
            raise ValueError(
                "Can't find a vocabulary file at path '{}'. To load the vocabulary from a Google pretrained "
                "model use `tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)`".format(vocab_file))
        self.do_lower_case = do_lower_case
        # 将Tag 内容增加至 never split中去：
        tokens_to_add = sorted(list(mapping.values()), key=lambda x: len(x), reverse=True)
        sorted_add_tokens = sorted(list(tokens_to_add), key=lambda x: len(x), reverse=True)
        self.never_split = never_split + tuple(sorted_add_tokens)
        # 对Bert字典进行添加
        self.vocab = load_vocab(vocab_file)
        orig_vocab_len = len(self.vocab)
        for idx, add_token in enumerate(sorted_add_tokens):
            self.vocab[add_token] = idx+orig_vocab_len
        
        # 对新的vocab生成字典用于映射
        self.ids_to_tokens = collections.OrderedDict(
            [(ids, tok) for tok, ids in self.vocab.items()])

        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case,
                                              never_split=self.never_split)
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)
        self.max_len = max_len if max_len is not None else int(1e12)

    def tokenize(self, text, use_bert_basic_tokenizer=False):
        split_tokens = []
        # 此部分工作的意义在于： 
        if(use_bert_basic_tokenizer):
            # 经过jieba分好词的结果组成的text
            pretokens = self.basic_tokenizer.tokenize(text)
        else:
            # 经过jieba分好词的结果
            pretokens = list(enumerate(text.split()))

        for i,token in pretokens:
            # if(self.do_lower_case):
            #     token = token.lower()
            subtokens = self.wordpiece_tokenizer.tokenize(token)
            for sub_token in subtokens:
                # 将分词后的bpe之后的结果进行整合
                split_tokens.append(sub_token)
        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            ids.append(self.vocab[token])
        # if len(ids) > self.max_len:
        #     raise ValueError(
        #         "Token indices sequence length is longer than the specified maximum "
        #         " sequence length for this BERT model ({} > {}). Running this"
        #         " sequence through BERT will result in indexing errors".format(len(ids), self.max_len)
        #     )
        return ids

    def convert_ids_to_tokens(self, ids):
        """Converts a sequence of ids in wordpiece tokens using the vocab."""
        tokens = []
        for i in ids:
            tokens.append(self.ids_to_tokens[i])
        return tokens

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, cache_dir=None, *inputs, **kwargs):
        """
        Instantiate a PreTrainedBertModel from a pre-trained model file.
        Download and cache the pre-trained model file if needed.
        """
        # if pretrained_model_name_or_path in PRETRAINED_VOCAB_ARCHIVE_MAP:
        #     vocab_file = PRETRAINED_VOCAB_ARCHIVE_MAP[pretrained_model_name_or_path]
        # else:
        vocab_file = pretrained_model_name_or_path
        if os.path.isdir(vocab_file):
            vocab_file = os.path.join(vocab_file, VOCAB_NAME)
        # redirect to the cache, if necessary
        # try:
        #     resolved_vocab_file = cached_path(vocab_file, cache_dir=cache_dir)
        # except EnvironmentError:
        #     logger.error(
        #         "Model name '{}' was not found in model name list ({}). "
        #         "We assumed '{}' was a path or url but couldn't find any file "
        #         "associated to this path or url.".format(
        #             pretrained_model_name_or_path,
        #             ', '.join(PRETRAINED_VOCAB_ARCHIVE_MAP.keys()),
        #             vocab_file))
        #     return None
        # if resolved_vocab_file == vocab_file:
        #     logger.info("loading vocabulary file {}".format(vocab_file))
        # else:
        #     logger.info("loading vocabulary file {} from cache at {}".format(
        #         vocab_file, resolved_vocab_file))
        if pretrained_model_name_or_path in PRETRAINED_VOCAB_POSITIONAL_EMBEDDINGS_SIZE_MAP:
            # if we're using a pretrained model, ensure the tokenizer wont index sequences longer
            # than the number of positional embeddings
            max_len = PRETRAINED_VOCAB_POSITIONAL_EMBEDDINGS_SIZE_MAP[pretrained_model_name_or_path]
            kwargs['max_len'] = min(kwargs.get('max_len', int(1e12)), max_len)
        # Instantiate tokenizer.
        tokenizer = cls(vocab_file, *inputs, **kwargs)
        return tokenizer


class BasicTokenizer(object):
    """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

    def __init__(self,
                 do_lower_case=True,
                 never_split=("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]")):
        """Constructs a BasicTokenizer.

        Args:
          do_lower_case: Whether to lower case the input.
        """
        self.do_lower_case = do_lower_case
        self.never_split = never_split

    def tokenize(self, text):
        """Tokenizes a piece of text."""
        text = self._clean_text(text)
        # This was added on November 1st, 2018 for the multilingual and Chinese
        # models. This is also applied to the English models now, but it doesn't
        # matter since the English models were not trained on any Chinese data
        # and generally don't have any Chinese data in them (there are Chinese
        # characters in the vocabulary because Wikipedia does have some Chinese
        # words in the English Wikipedia.).
        text = self._tokenize_chinese_chars(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for i,token in enumerate(orig_tokens):
            if self.do_lower_case and token not in self.never_split:
                token = token.lower()
                token = self._run_strip_accents(token)
            # split_tokens.append(token)
            split_tokens.extend([(i,t) for t in self._run_split_on_punc(token)])

        # output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return split_tokens

    def _run_strip_accents(self, text):
        """Strips accents from a piece of text."""
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text):
        """Splits punctuation on a piece of text."""
        if text in self.never_split:
            return [text]
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]

    def _tokenize_chinese_chars(self, text):
        """Adds whitespace around any CJK character."""
        output = []
        for char in text:
            cp = ord(char)
            if self._is_chinese_char(cp):
                output.append(" ")
                output.append(char)
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)

    def _is_chinese_char(self, cp):
        """Checks whether CP is the codepoint of a CJK character."""
        # This defines a "chinese character" as anything in the CJK Unicode block:
        #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
        #
        # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
        # despite its name. The modern Korean Hangul alphabet is a different block,
        # as is Japanese Hiragana and Katakana. Those alphabets are used to write
        # space-separated words, so they are not treated specially and handled
        # like the all of the other languages.
        if ((cp >= 0x4E00 and cp <= 0x9FFF) or  ##
                (cp >= 0x3400 and cp <= 0x4DBF) or  ##
                (cp >= 0x20000 and cp <= 0x2A6DF) or  ##
                (cp >= 0x2A700 and cp <= 0x2B73F) or  ##
                (cp >= 0x2B740 and cp <= 0x2B81F) or  ##
                (cp >= 0x2B820 and cp <= 0x2CEAF) or  
                (cp >= 0xF900 and cp <= 0xFAFF) or  ##
                (cp >= 0x2F800 and cp <= 0x2FA1F)):  ##
            return True

        return False

    def _clean_text(self, text):
        """Performs invalid character removal and whitespace cleanup on text."""
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)


class WordpieceTokenizer(object):
    """Runs WordPiece tokenization."""

    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def tokenize(self, text):
        """Tokenizes a piece of text into its word pieces.

        This uses a greedy longest-match-first algorithm to perform tokenization
        using the given vocabulary.

        For example:
          input = "unaffable"
          output = ["un", "##aff", "##able"]

        Args:
          text: A single token or whitespace separated tokens. This should have
            already been passed through `BasicTokenizer`.

        Returns:
          A list of wordpiece tokens.
        """

        output_tokens = []
        for token in whitespace_tokenize(text):
            chars = list(token)
            if len(chars) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            is_bad = False
            start = 0
            sub_tokens = []
            while start < len(chars):
                end = len(chars)
                cur_substr = None
                while start < end:
                    substr = "".join(chars[start:end])
                    if start > 0:
                        substr = "##" + substr
                    if substr in self.vocab:
                        cur_substr = substr
                        break
                    end -= 1
                if cur_substr is None:
                    is_bad = True
                    break
                sub_tokens.append(cur_substr)
                start = end

            if is_bad:
                output_tokens.append(self.unk_token)
            else:
                output_tokens.extend(sub_tokens)
        return output_tokens


def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False


In [16]:
tag_mapping = {'我操了':'<<我操了>>'}

tokenizer = BertTokenizer.from_pretrained('../../pretrained/bert_base_chinese', do_lower_case=True, mapping=tag_mapping)

In [17]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(''))

[21128]

In [32]:
a = tokenizer.tokenize('我 真 是 <<我操了>> [unused99]')
a

['我', '真', '是', '<<我操了>>', '[unused99]']

In [33]:
tokenizer.convert_tokens_to_ids(a)

[2769, 4696, 3221, 21128, 99]

In [36]:
topic_utter_ids = [1,3,6,9,12]
queue = []
start_ids = []
end_ids = []
for i, idx in enumerate(topic_utter_ids):
    # 初始化第一个元素：
    if i == 0: 
        queue.append(idx)
    else:
        if queue[-1]+1 == idx:
            queue.append(idx)
        else:
            start_ids.append(queue[0])
            end_ids.append(queue[-1])
            queue = []
            queue.append(idx)
start_ids.append(queue[0])
end_ids.append(queue[-1])

In [37]:
start_ids,end_ids

([1, 3, 6, 9, 12], [1, 3, 6, 9, 12])

In [38]:
a = [1,2,3]
b = [1,2,4]

In [40]:
set(a) | set(b)

{1, 2, 3, 4}

In [11]:
import json
with open('../data/CSDS/dict/label2tag.json','r',encoding = 'utf-8') as js_object:
    a = json.load(js_object)

In [12]:
a

{'联系售后': '<<联系售后>>',
 '商品介绍': '<<商品介绍>>',
 '商品检索': '<<商品检索>>',
 '返回方式': '<<返回方式>>',
 '自营与第三方区别': '<<自营与第三方区别>>',
 '话费充值': '<<话费充值>>',
 '售后维修点查询': '<<售后维修点查询>>',
 '支付密码': '<<支付密码>>',
 '查看发票': '<<查看发票>>',
 '联系客服': '<<联系客服>>',
 '联系商家': '<<联系商家>>',
 '京豆积分扣除': '<<京豆积分扣除>>',
 '填写发票信息': '<<填写发票信息>>',
 '取消订单受理时间': '<<取消订单受理时间>>',
 '服务单修改': '<<服务单修改>>',
 '支付到账时间': '<<支付到账时间>>',
 '是否全国联保': '<<是否全国联保>>',
 '是否送货上门': '<<是否送货上门>>',
 '安装收费': '<<安装收费>>',
 '代客户充值': '<<代客户充值>>',
 '为什么显示无货': '<<为什么显示无货>>',
 '订单回收站': '<<订单回收站>>',
 '评价晒单异常': '<<评价晒单异常>>',
 '预约抢购': '<<预约抢购>>',
 '下单后无记录': '<<下单后无记录>>',
 '预售订单': '<<预售订单>>',
 '其他': '<<其他>>',
 '小金库转入转出': '<<小金库转入转出>>',
 '查询个人资料': '<<查询个人资料>>',
 '返修退换货审核不通过': '<<返修退换货审核不通过>>',
 '京豆积分比值': '<<京豆积分比值>>',
 '白条分期手续费': '<<白条分期手续费>>',
 '京豆积分解释': '<<京豆积分解释>>',
 '京东钱包绑定': '<<京东钱包绑定>>',
 '恢复订单': '<<恢复订单>>',
 '如何评价晒单': '<<如何评价晒单>>',
 '白条还款方式': '<<白条还款方式>>',
 '京豆积分查看': '<<京豆积分查看>>',
 '近期活动咨询': '<<近期活动咨询>>',
 '有什么颜色': '<<有什么颜色>>',
 '申请退款': '<<申请退款>>',
 '账户注册': '<<账户注册>>',
 '

In [2]:
import torch 
x = torch.tensor([[[1]]])
assert x.size() == (1,1,1)

In [13]:

sents_vec = torch.tensor([[[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],[[0,1,1,1],[1,2,2,2],[2,3,3,3],[3,4,4,4]]])
src_tokens_index = torch.tensor([[0,1,2,0,0,0],[3,1,2,0,1,0]])
all = []
for i in range(sents_vec.size(0)):
    all.append(torch.index_select(sents_vec[i], 0, src_tokens_index[i]))
# b, tgt_len, h
torch.stack(all)
# sents_vec.gather(index=src_tokens_index.unsqueeze(-1), dim=1)

tensor([[[1, 1, 1, 1],
         [2, 2, 2, 2],
         [3, 3, 3, 3],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]],

        [[3, 4, 4, 4],
         [1, 2, 2, 2],
         [2, 3, 3, 3],
         [0, 1, 1, 1],
         [1, 2, 2, 2],
         [0, 1, 1, 1]]])

In [14]:
topic_tgt = torch.tensor([[1,2,3,4,56]])
topic_tgt.masked_fill(topic_tgt.ge(10), 0)

tensor([[1, 2, 3, 4, 0]])

In [25]:
a=torch.tensor([True,False]).unsqueeze(-1)
# a.unsqueeze(-1)
b=torch.tensor([[1,2],[1,3]])

c=torch.tensor([[0,0],[0,0]])
# b=torch.tensor([[1,2],[3,4]])
# c=torch.tensor([[0,0],[0,0]])
torch.where(a, b, c)

tensor([[1, 2],
        [0, 0]])

In [11]:
import torch 
c=torch.tensor([[False,True],[True,True]])
c.sum().item()

3

In [14]:
outputs = torch.tensor([[1,2,3],[1,2,4],[1,2,10]])
outputs.max(1)

torch.return_types.max(
values=tensor([ 3,  4, 10]),
indices=tensor([2, 2, 2]))

In [17]:
pred = torch.tensor([1,2,3,4,5,6,7])
gold = torch.tensor([1,2,3,3,3,3,3])
non_padding = torch.tensor([False,True,True,False,False,False,True])
pred.eq(gold).masked_select(non_padding)

tensor([ True,  True, False])

In [21]:
assert pred.size() == gold.size()

In [28]:
float(pred.numel())

7.0

In [29]:
pred.max()

tensor(7)

In [1]:
import torch 
mask = torch.tensor([[True,True,False]]).unsqueeze(1)
word_scores = torch.tensor([[[1,2,3],[1,2,3],[1,2,3]]])
word_scores.masked_fill(mask, 100)

tensor([[[100, 100,   3],
         [100, 100,   3],
         [100, 100,   3]]])

In [1]:
import torch 
torch.arange(10)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
torch.tensor([1,1]).repeat(10)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [5]:
torch.full([3,1],10)[:,-1].view(1, -1)

tensor([[10, 10, 10]])

In [10]:
import json
with open('/home/sda/hanqinyu/DPC/RODS-main/models/BERT_interact/data/CSDS/dict/label2vid.json','r',encoding = 'utf-8') as js_object:
    label2vid = json.load(js_object) 

In [16]:
import torch 
a = torch.tensor([[1,2,3],[1,2,4],[1,2,5]])
id = torch.tensor([0,2,1])
a[id]

tensor([[1, 2, 3],
        [1, 2, 5],
        [1, 2, 4]])

In [19]:
import torch.nn.functional as F
a = torch.tensor([[[1,2,3],[1,2,4],[1,2,5]]])
b = torch.tensor([[1,1,1],[1,2,3]])
F.linear(a,b)

tensor([[[ 6, 14],
         [ 7, 17],
         [ 8, 20]]])

In [3]:
import torch 
sequence = torch.tensor([[1,1,1],[1,2,3]])
sequence.sum(dim=0) / (sequence != 0).sum(dim=0)
# (sequence != 0).sum(dim=0)

tensor([1.0000, 1.5000, 2.0000])

In [4]:
import torch.nn as nn
embedding = nn.Embedding(10, 3)

In [7]:
embedding.weight.data

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.5265,  1.2232, -0.5563],
        [-0.7228,  0.7736, -0.8997],
        [-0.3493,  0.3765, -1.8596],
        [-0.5221,  0.0956, -1.7240],
        [-1.0532, -0.0440, -1.3779],
        [-0.2831,  0.6198,  1.0566],
        [-0.4747, -0.0486,  0.8864],
        [-0.5696,  1.4773, -0.3870],
        [ 0.3113,  0.7729, -0.1542]])

In [6]:
embedding = nn.Embedding(10, 3,padding_idx=0)

In [8]:
a = nn.Dropout(0.2)

In [9]:
a(embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.6581,  1.5289, -0.0000],
        [-0.9035,  0.9670, -1.1246],
        [-0.4366,  0.4706, -2.3245],
        [-0.6526,  0.1195, -2.1550],
        [-1.3165, -0.0550, -1.7223],
        [-0.3539,  0.0000,  1.3207],
        [-0.5933, -0.0000,  1.1080],
        [-0.0000,  1.8467, -0.0000],
        [ 0.0000,  0.9662, -0.1927]])

In [1]:
import torch
word_scores = torch.tensor([[[1,2,3],[1,2,3],[1,2,3]]])
mask = torch.tensor([[True,False,True]]).unsqueeze(1)
word_scores.masked_fill(mask, 100)

tensor([[[100,   2, 100],
         [100,   2, 100],
         [100,   2, 100]]])

In [2]:

a = torch.tensor([[1,2,3],[1,2,3],[1,2,3]])
a.max(1)[1]

tensor([2, 2, 2])

In [4]:
a = enumerate([0,1,2,3,4,5,6,7,8])
for idx, topic_id in a:
    print(topic_id)
    if topic_id == 2:
        next(a)
        next(a)
        next(a)


0
1
2
6
7
8


In [20]:
tgt_fin_role = []
tgt_fin_token_idxs = [1,2,3,4,5,6]
shift = True
for idx in tgt_fin_token_idxs:
    if shift:
        tgt_fin_role.append(1)
    else:
        tgt_fin_role.append(2)
    if idx in [3,5]:
        shift = not shift

In [21]:
tgt_fin_role

[1, 1, 1, 2, 2, 1]

In [9]:
import torch 
a = torch.tensor([[1,2,3,4],[1,2,3,5],[1,2,3,6]])
b = torch.tensor([[1,2,3,4],[1,2,3,5],[1,2,3,6]])

In [7]:
a.device

device(type='cpu')

In [10]:
torch.stack([a,b])


tensor([[[1, 2, 3, 4],
         [1, 2, 3, 5],
         [1, 2, 3, 6]],

        [[1, 2, 3, 4],
         [1, 2, 3, 5],
         [1, 2, 3, 6]]])

In [4]:
import torch 
a = torch.tensor([[[1,2,3,4],[1,2,3,5],[1,2,3,6]]])


In [3]:
if a is not None:
    print('EE')

EE


In [5]:
a.sum(2, keepdim=True)

tensor([[[10],
         [11],
         [12]]])

In [6]:
a.log()

tensor([[[0.0000, 0.6931, 1.0986, 1.3863],
         [0.0000, 0.6931, 1.0986, 1.6094],
         [0.0000, 0.6931, 1.0986, 1.7918]]])

In [7]:
a = torch.tensor([[[1,2,3,4],[1,2,3,5],[1,2,3,6]]])
b = torch.tensor([[[1,2,3,4],[1,2,3,5],[1,2,3,6]]])

torch.sigmoid(a)

tensor([[[0.7311, 0.8808, 0.9526, 0.9820],
         [0.7311, 0.8808, 0.9526, 0.9933],
         [0.7311, 0.8808, 0.9526, 0.9975]]])

In [8]:
a + 1

tensor([[[2, 3, 4, 5],
         [2, 3, 4, 6],
         [2, 3, 4, 7]]])

In [14]:
import torch 
a = torch.tensor([[[[1,2,3,4],[1,2,3,5],[1,2,3,6]],[[1,2,3,13],[1,2,3,14],[1,2,3,15]]]])
b = torch.tensor([[[[2],[3],[4]]]])

c = torch.ones(10)
c[1:8] = 10

In [15]:
c

tensor([ 1., 10., 10., 10., 10., 10., 10., 10.,  1.,  1.])

In [5]:
import torch 
a = torch.tensor([[[1,2,3,4],[1,2,3,5],[1,2,3,6]]]).float()
torch.zeros(a.size())

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [9]:
b = torch.tensor([[[1,2,0,4],[1,2,3,0],[1,0,3,6]]]).bool()

In [10]:
~b

tensor([[[False, False,  True, False],
         [False, False, False,  True],
         [False,  True, False, False]]])

In [6]:
a-2

tensor([[[-1.,  0.,  1.,  2.],
         [-1.,  0.,  1.,  3.],
         [-1.,  0.,  1.,  4.]]])

In [13]:
b = torch.zeros(1,4)
for i in range(3):
    b += a[:,i,:]
    a[:,i,:] = a[:,i,:]/b
    print(b)
# # a[:,1,:] = a[:,1,:]
print(a)

tensor([[1., 2., 3., 4.]])
tensor([[2., 4., 6., 9.]])
tensor([[ 3.,  6.,  9., 15.]])
tensor([[[1.0000, 1.0000, 1.0000, 1.0000],
         [0.5000, 0.5000, 0.5000, 0.5556],
         [0.3333, 0.3333, 0.3333, 0.4000]]])


In [18]:
a = torch.tensor([[1,2,3,4],[1,2,3,5],[1,2,3,6]])
b = torch.tensor([[1,2,3,3],[1,2,3,4],[1,2,3,20]])
torch.stack([a,b],1)

tensor([[[ 1,  2,  3,  4],
         [ 1,  2,  3,  3]],

        [[ 1,  2,  3,  5],
         [ 1,  2,  3,  4]],

        [[ 1,  2,  3,  6],
         [ 1,  2,  3, 20]]])

In [41]:
a = torch.tensor([[[1,2,3,4],[1,2,3,5],[1,2,3,6]]])
b = torch.tensor([2,3,4])
c = torch.tensor([[[1,1,1,1],[1,1,1,1],[1,1,1,20]]])



In [5]:
a = torch.tensor([[True,True,False],[True,False,False]])
a.sum(1)

tensor([2, 1])

In [35]:
torch.softmax(a,-1)

tensor([0.5000, 0.5000, 0.0000])

In [22]:
torch.zeros(1,2).fill_(1e-10)

tensor([[1.0000e-10, 1.0000e-10]])

In [2]:
import torch 
from torch.nn.utils.rnn import pad_sequence
a = torch.tensor([[1,2,3,4],[1,2,3,5],[1,2,3,6]])
b = torch.tensor([1,2,3,4])
pad_sequence([a,b],batch_first = True,padding_value = 0)

tensor([[[ 1,  2,  3,  4],
         [ 1,  2,  3,  5],
         [ 1,  2,  3,  6]],

        [[ 1,  2,  3,  4],
         [ 1,  2,  3, 20],
         [ 0,  0,  0,  0]]])

In [8]:

torch.arange(1,9)

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [14]:
a=torch.cat([torch.arange(1,12),torch.tensor([0])])
b=torch.cat([torch.arange(1,8),torch.tensor([0])])

In [16]:
tag_tgt=pad_sequence([a,b],batch_first=True,padding_value=-1)

In [17]:
tag_tgt[tag_tgt == -1] = 0

In [18]:
tag_tgt

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0],
        [ 1,  2,  3,  4,  5,  6,  7,  0,  0,  0,  0,  0]])

In [19]:
torch.empty(1, 3)

tensor([[-3.0237e-18,  3.0677e-41, -3.2979e-18]])

In [20]:
torch.empty([1, 3])

tensor([[-4.3708e-18,  3.0677e-41, -3.4086e-18]])

In [22]:
a = torch.tensor([[2,2,3,4],[1,2,3,5],[1,2,3,6]])
a.eq(2).sum()

tensor(4)

In [23]:
a = torch.tensor([[2,2,3,4],[1,2,3,5],[1,2,3,6]])
b = torch.tensor([True,True,False])
torch.masked_select(a,b.unsqueeze(-1))

tensor([2, 2, 3, 4, 1, 2, 3, 5])

In [37]:
import numpy as np
shuffle_index = [i for i in range(7)]
np.random.shuffle(shuffle_index)

In [38]:
shuffle_index

[6, 2, 1, 5, 4, 0, 3]

In [29]:
torch.arange(1,8)[shuffle_index]

tensor([5, 4, 6, 2, 7, 1, 3])

In [30]:
np.argsort(shuffle_index)

array([5, 3, 6, 1, 0, 2, 4])

In [32]:
a = torch.tensor([[2,2,3,4],[1,2,3,5],[1,2,3,6]])
a[[1,2,0]]

tensor([[1, 2, 3, 5],
        [1, 2, 3, 6],
        [2, 2, 3, 4]])

In [39]:
for id in range(0):
    prine(id)

In [41]:
seg_i = [1,2,6,8,0]
seggram = [0,3,4,2]
len(set(seg_i) & set(seggram))

2

In [5]:
def change_word2id(ref, pred):
    # ref = re.sub(' ', '', ref)
    # pred = re.sub(' ', '', pred)
    # pred = re.sub('<q>', '', pred)
    ref_id, pred_id = [], []
    tmp_dict = {}
    new_index = 0
    words = list(ref)
    for w in words:
        if w not in tmp_dict.keys():
            tmp_dict[w] = new_index
            ref_id.append(str(new_index))
            new_index += 1
        else:
            ref_id.append(str(tmp_dict[w]))
    words = list(pred)
    for w in words:
        if w not in tmp_dict.keys():
            tmp_dict[w] = new_index
            pred_id.append(str(new_index))
            new_index += 1
        else:
            pred_id.append(str(tmp_dict[w]))
    return ' '.join(ref_id), ' '.join(pred_id)

In [3]:
import re
ref, pred = [ 0,  4,  8, 28,  2,  1], [ 4, 12, 14, 15, 16, 18,  2, 24, 27,  2, 29, 33,  2, 31,  2,  1]
# change_word2id(ref, pred)
change_word2id_split(ref, pred, True)

('1 2 3 4 5 6', '2 7 8 9 10 11 5 12 13 5 14 15 5 16 5 6')

In [6]:
change_word2id(ref, pred)

('0 1 2 3 4 5', '1 6 7 8 9 10 4 11 12 4 13 14 4 15 4 5')

In [2]:
def change_word2id_split(ref, pred, num):
    if num:
        pass
    else:
        ref = re.sub(' ', '', ref)
        pred = re.sub(' ', '', pred)
        ref = re.sub('<q>', '', ref)
        pred = re.sub('<q>', '', pred)
    ref_id, pred_id = [], []
    tmp_dict = {'%': 0}
    new_index = 1
    words = list(ref)
    for w in words:
        if w not in tmp_dict.keys():
            tmp_dict[w] = new_index
            ref_id.append(str(new_index))
            new_index += 1
        else:
            ref_id.append(str(tmp_dict[w]))
        if w == '。':
            ref_id.append(str(0))
    words = list(pred)
    for w in words:
        if w not in tmp_dict.keys():
            tmp_dict[w] = new_index
            pred_id.append(str(new_index))
            new_index += 1
        else:
            pred_id.append(str(tmp_dict[w]))
        if w == '。':
            pred_id.append(str(0))
    return ' '.join(ref_id), ' '.join(pred_id)

In [1]:
a = [1,2,3]

In [2]:
if a is not None:
    print("!!!")

!!!


In [1]:
value = "不详。"
if value.strip().strip('。') in ['暂无','暂缺','无','不详']:
    print('@@@@@@@@@@@@')

@@@@@@@@@@@@


In [2]:
if "" in ['']:
    print('WW')

WW


In [5]:
a = ''
if a.strip().strip('。') in ['']:
    print('WW')

WW


In [2]:
import torch
from torch import autograd
input = autograd.Variable(torch.randn(3,3), requires_grad=True)
print(input)

tensor([[-0.9477,  0.4474, -0.7866],
        [-0.8812,  0.9712, -0.0094],
        [-2.6336, -0.4913,  0.8207]], requires_grad=True)


In [3]:
from torch import nn
sigmoid= nn.Sigmoid()
print(sigmoid(input))

tensor([[0.2793, 0.6100, 0.3129],
        [0.2929, 0.7254, 0.4976],
        [0.0670, 0.3796, 0.6944]], grad_fn=<SigmoidBackward0>)


In [4]:
label = torch.FloatTensor([[[0, 1, 1], [1, 1, 1], [0, 0, 0]]])
input = input.unsqueeze(0)

In [6]:
loss = nn.BCELoss(reduction='none')
print(loss(sigmoid(input), label))

tensor([[[0.3276, 0.4943, 1.1619],
         [1.2278, 0.3211, 0.6979],
         [0.0694, 0.4774, 1.1855]]], grad_fn=<BinaryCrossEntropyBackward0>)


In [12]:
import torch

a = torch.tensor([[1,2,0,0,3],[1,0,0,0,3]]).bool()
a&a

tensor([[ True,  True, False, False,  True],
        [ True, False, False, False,  True]])

In [19]:
mask_cls = torch.tensor([[True,False]])
np.where(mask_cls.cpu().data.numpy()[0] > 0)

(array([0]),)

In [17]:
for j, label in enumerate(intent_label[0]):
    if label == 1:
        print(label)

tensor(1)


In [20]:
a = '123333'
list(a)

['1', '2', '3', '3', '3', '3']

In [24]:
from cal_rouge import cal_rouge_path
cal_rouge_path('/home/sda/hanqinyu/DPC/RODS-main/models/BERT_interact_former/logs/bert_both_ft__test_.1500.candidate', '/home/sda/hanqinyu/DPC/RODS-main/models/BERT_interact_former/logs/bert_both_ft__test_.1500.gold')

ModuleNotFoundError: No module named 'others'